
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment - Constraint Satisfaction Problem <br>
<font color=696880 size=4>


____

## Sudoku Solver : Backtracking
Use your knowledge of the backtracking algorithm to solve the sudoku problem

### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

### Preprocessing
Import the data and explore it!
The columns you need are named quizzes and solutions
Unsolved sudoku are in quizzes and solved ones are in solutions, use the solutions column to check whether you solve the sudoku correctly

In [ ]:
data = pd.read_csv('sudoku.csv')
print(data.head())

   Unnamed: 0                                            quizzes  \
0           0  0043002090050090010700600430060020871900074000...   
1           1  0401000501070039605200080000000000170009068008...   
2           2  6001203840084590720000060050002640300700800069...   
3           3  4972000001004000050000160986203000403009000000...   
4           4  0059103080094030600275001000300002010008200070...   

                                           solutions  
0  8643712593258497619712658434361925871986574322...  
1  3461792581875239645296483719658324174729168358...  
2  6951273841384596727248369158512647392739815469...  
3  4972583161864397252537164986293815473759641828...  
4  4659123781894735623275681497386452919548216372...  


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2000 non-null   int64 
 1   quizzes     2000 non-null   object
 2   solutions   2000 non-null   object
dtypes: int64(1), object(2)
memory usage: 47.0+ KB
None


In [ ]:
print(data.describe())

        Unnamed: 0
count  2000.000000
mean    999.500000
std     577.494589
min       0.000000
25%     499.750000
50%     999.500000
75%    1499.250000
max    1999.000000


In [ ]:
def gridding_array(array):
    return np.array(list(map(int, list(array)))).reshape((9, 9)) #1D array

In [ ]:
def display_grid(grid):
    """
    This function prints a formatted representation of the 9x9 Sudoku grid with dividing lines.
    """
    separator_row = "+-----+-------+-----+"

    for row in range(9):
        if row % 3 == 0:
            print(separator_row)

        row_content = []
        for col in range(9):
            row_content.append(str(grid[row, col]))
            if (col + 1) % 3 == 0 and col != 8:
                row_content.append("|")

        print(" ".join(row_content))

    print(separator_row)


### Backtracking Algorithm
Testing your algorithm on a sample sudoku is enough, but your code will be manually tested on multiple ones later

In [ ]:
def is_empty(sudoku):
    return np.any(sudoku == 0)

In [ ]:
def is_placement_valid(grid, num, position):
    row, col = position
    if num in grid[row, :]: # Checking row
        return False
    if num in grid[:, col]: # Checking column
        return False
    box_row, box_col = row // 3 * 3, col // 3 * 3
    if num in grid[box_row:box_row+3, box_col:box_col+3]: # Checking 3x3 box
        return False
    return True

In [ ]:
def backtrack(sudoku):
    if not is_empty(sudoku):
        return True
    for row in range(9):
        for col in range(9):
            if sudoku[row, col] == 0:  # Finding empty cells
                for num in range(1, 10):  # Try 1,2,..,9
                    if is_placement_valid(sudoku, num, (row, col)):
                        sudoku[row, col] = num
                        if backtrack(sudoku):
                            return True
                        sudoku[row, col] = 0
                return False  # No valid number
    return False

In [ ]:
def compare(sudoku, solution):
    diff = sudoku - solution
    return np.all(diff == 0)

sample_sudoku = gridding_array(data['quizzes'][0])
sample_solution = gridding_array(data['solutions'][0])
print("before solving:")
display_grid(sample_sudoku)
backtrack(sample_sudoku)
print("after solving:")
display_grid(sample_sudoku)
print("Correct Solution:", compare(sample_sudoku, sample_solution))


before solving:
+-----+-------+-----+
0 0 4 | 3 0 0 | 2 0 9
0 0 5 | 0 0 9 | 0 0 1
0 7 0 | 0 6 0 | 0 4 3
+-----+-------+-----+
0 0 6 | 0 0 2 | 0 8 7
1 9 0 | 0 0 7 | 4 0 0
0 5 0 | 0 8 3 | 0 0 0
+-----+-------+-----+
6 0 0 | 0 0 0 | 1 0 5
0 0 3 | 5 0 8 | 6 9 0
0 4 2 | 9 1 0 | 3 0 0
+-----+-------+-----+
after solving:
+-----+-------+-----+
8 6 4 | 3 7 1 | 2 5 9
3 2 5 | 8 4 9 | 7 6 1
9 7 1 | 2 6 5 | 8 4 3
+-----+-------+-----+
4 3 6 | 1 9 2 | 5 8 7
1 9 8 | 6 5 7 | 4 3 2
2 5 7 | 4 8 3 | 9 1 6
+-----+-------+-----+
6 8 9 | 7 3 4 | 1 2 5
7 1 3 | 5 2 8 | 6 9 4
5 4 2 | 9 1 6 | 3 7 8
+-----+-------+-----+
Correct Solution: True


## Sudoku Solver : CP-SAT Solver
Now we are going to explore ortools library and use it to solve the sudoku problem
Hint: use built-in functions of this library like AddAllDifferent and CpSolver; search for more ...
here is a useful link : https://developers.google.com/optimization/cp

In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
import numpy as np

In [ ]:
def solve_sudoku(sudoku):
    model = cp_model.CpModel()
    cells = {}
    for i in range(9):
        for j in range(9):
            cells[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')

    for i in range(9): # Row constraints
        model.AddAllDifferent([cells[(i, j)] for j in range(9)])

    for j in range(9): # Column constraints
        model.AddAllDifferent([cells[(i, j)] for i in range(9)])

    # Box constraints
    for box_row in range(3):
        for box_col in range(3):
            model.AddAllDifferent([cells[(box_row * 3 + i, box_col * 3 + j)] for i in range(3) for j in range(3)])

    for i in range(9):
        for j in range(9):
            if sudoku[i, j] != 0:
                model.Add(cells[(i, j)] == sudoku[i, j])

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        solved_grid = np.zeros((9, 9), dtype=int)
        for i in range(9):
            for j in range(9):
                solved_grid[i, j] = solver.Value(cells[(i, j)])
        return solved_grid
    return None


In [ ]:
solved_sudoku = solve_sudoku(sample_sudoku)
display_grid(solved_sudoku)
print("Correct Solution:", compare(solved_sudoku, sample_solution))

+-----+-------+-----+
8 6 4 | 3 7 1 | 2 5 9
3 2 5 | 8 4 9 | 7 6 1
9 7 1 | 2 6 5 | 8 4 3
+-----+-------+-----+
4 3 6 | 1 9 2 | 5 8 7
1 9 8 | 6 5 7 | 4 3 2
2 5 7 | 4 8 3 | 9 1 6
+-----+-------+-----+
6 8 9 | 7 3 4 | 1 2 5
7 1 3 | 5 2 8 | 6 9 4
5 4 2 | 9 1 6 | 3 7 8
+-----+-------+-----+
Correct Solution: True


## Graph Coloring
In this part, we are going to explore the graph coloring topic. Now we can use advanced options to speed up the process of constraint satisfaction

In [ ]:
from itertools import combinations
from time import time
import numpy as np
import copy

### AC-3 Algorithm
First and foremost, implement the AC-3 algorithm to be later used in the problem

In [ ]:
def AC3(csp, neighbors):
    queue = [(xi, xj) for (xi, xj) in csp['constraints'].keys()]
    while queue:
        (xi, xj) = queue.pop(0)
        if revise(csp, xi, xj):
            if len(csp['domains'][xi]) == 0:
                return False, None
            for xk in neighbors[xi]:
                queue.append((xk, xi))
    return True, csp['domains']

def revise(csp, xi, xj):
    revised = False
    for value in csp['domains'][xi]:
        if not any(satisfies_constraint(value, y, xi, xj, csp['constraints']) for y in csp['domains'][xj]):
            csp['domains'][xi].remove(value)
            revised = True
    return revised

def satisfies_constraint(xi_value, xj_value, xi, xj, constraints):
    return (xi_value, xj_value) in constraints[(xi, xj)] #Check if the constraint between xi and xj is satisfied by the values assigned to them

### AC-4 Algorithm
Secondly, implement the AC-4 (Arc Consistency 4) algorithm. This algorithm enforces arc consistency by maintaining **support tables** and efficiently processing **a queue of arcs** for consistency checks. To help you implement it, here are the key steps and concepts to focus on:
1. **Support Tables**: For each pair of variable-value assignments, you must maintain a table that tracks the number of supporting values in the neighboring variables' domains. This table will allow you to quickly determine if a value is consistent with its neighbors
2. Start by initializing your support tables for all variable-value pairs. For each arc (X, Y), compute and store the number of values in Y's domain that support each value in X's domain based on the constraint between X and Y.
3. Initialize **a queue of arcs** with all pairs of variables that share a constraint
4. For each arc (X, Y) in the queue, check the support for each value in X's domain. If any value in X loses all supporting values in Y, remove that value from X’s domain.
5. If a value is removed from X’s domain, for every neighboring variable Z (other than Y), you must add the arc (Z, X) back into the queue for further consistency checks. This ensures that removing a value from X doesn’t create inconsistencies for other variables.
6. Whenever a value is removed from a domain, update the support tables for all neighboring variable-value pairs that depend on the removed value. This way, if a value of Z is only supported by the value that was removed from X, the support count for that value of Z should be decremented.
7. The algorithm terminates when the queue is empty, meaning all arcs have been processed and the domains of all variables are arc-consistent. If at any point a domain becomes empty, the problem is unsolvable.

In [ ]:
def AC4(csp):
    support = {xi: {value: 0 for value in csp['domains'][xi]} for xi in csp['nodes']}

    for (xi, xj) in csp['constraints'].keys():
        for value in csp['domains'][xi]:
            if any(satisfies_constraint(value, y, xi, xj, csp['constraints']) for y in csp['domains'][xj]):
                support[xi][value] += 1

    queue = [(xi, xj) for (xi, xj) in csp['constraints'].keys()]
    while queue:
        (xi, xj) = queue.pop(0)
        for value in csp['domains'][xi][:]:
            if support[xi][value] == 0:
                csp['domains'][xi].remove(value)
                for xk in neighbors[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
                        for y in csp['domains'][xk]:
                            if satisfies_constraint(value, y, xi, xj, csp['constraints']):
                                support[xk][y] -= 1
    return True, csp['domains']

In [ ]:
graph = {
    "WA": ["NT", "SA"],
    "NT": ["WA", "SA", "Q"],
    "SA": ["NT", "WA", "Q", "NSW", "V"],
    "Q": ["NT", "SA", "NSW"],
    "NSW": ["Q", "SA", "V"],
    "V": ["NSW", "SA"]
}

nodes = ["WA", "NT", "SA", "Q", "NSW", "V"]
domains = {}

for node in nodes:
    if node == "WA":
        domains[node] = ["R"]
    else:
        domains[node] = ["R", "G", "B"]

constraint = [(x, y) for x in ["R", "G", "B"] for y in ["R", "G", "B"] if x != y]
constraints = {}

for node in graph.keys():
    for neighbor in graph[node]:
        constraints[(node, neighbor)] = constraint

csp = {
    "nodes": nodes,
    "domains": domains,
    "constraints": constraints
}

#AC3
start = time()
is_consistent_ac3, reduced_domains_ac3 = AC3(csp, graph)
s_ac3 = time() - start
if is_consistent_ac3:
    print("Domains after AC3:\n", reduced_domains_ac3)
else:
    print("Inconsistent!")

print("Time taken for AC3: %.4g seconds" % s_ac3)

#AC4
start = time()
is_consistent_ac4, reduced_domains_ac4 = AC4(csp)
s_ac4 = time() - start
if is_consistent_ac4:
    print("Domains after AC4:\n", reduced_domains_ac4)
else:
    print("Inconsistent!")

print("Time taken for AC4: %.4g seconds" % s_ac4)

Domains after AC3:
 {'WA': ['R'], 'NT': ['G', 'B'], 'SA': ['G', 'B'], 'Q': ['R', 'G', 'B'], 'NSW': ['R', 'G', 'B'], 'V': ['R', 'G', 'B']}
Time taken for AC3: 0 seconds
Domains after AC4:
 {'WA': ['R'], 'NT': ['G', 'B'], 'SA': ['G', 'B'], 'Q': ['R', 'G', 'B'], 'NSW': ['R', 'G', 'B'], 'V': ['R', 'G', 'B']}
Time taken for AC4: 0.0009964 seconds


### Conclusion
Explain the differences between these two algorithms in terms of time and space complexity. Which one do you prefer to use?

**Your answer**:

Time Complexity : The worst-case time complexity of $AC-3$ is $O(ed^3)$ and for $AC-4$ is $O(ed^2)$, where e is the number of edges (constraints) in the constraint graph and d is the maximum size of the domain of any variable. $AC-4$ is more efficient than $AC-3$ in that it typically avoids re-checking previously validated values by maintaining a table of allowed values for each variable.

Space Complexity : The space complexity of $AC-3$ is $O(e + d)$ and for $AC-4$ is $O(d^2 + e)$. $AC-3$ primarily uses a queue to keep track of the arcs to be checked, which can grow based on the number of constraints. $AC-4$ may require more space to store information about allowed pairs but can be more efficient in terms of runtime due to reduced re-checking of values.

Then, $AC-4$ is generally more efficient in terms of time complexity, especially in cases where the domain sizes are large. However, it has a higher space requirement due to the need for maintaining additional data structures. The choice between $AC-3$ and $AC-4$ may depend on the specific constraints and size of the problem being addressed, as well as the available memory resources.


## Backtracking + AC-3 and AC-4 algorithm
Now we are going to utilize the implemented algorithms to speed up the backtracking process, use both of the above algorithms, you must run ac3 and ac4 algorithms seperately.

In [ ]:
def is_consistent(var, value, assignment, csp, neighbors):
    for neighbor in neighbors[var]: #checking consistency
        if neighbor in assignment and not satisfies_constraint(value, assignment[neighbor], var, neighbor, csp['constraints']):
            return False
    return True

def select_unassigned_variable(assignment, domains): #MRV
    unassigned = [v for v in domains.keys() if v not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))

def backtracking_search(csp, neighbors, ac3):
    assignment = {}

    def backtrack():
        if len(assignment) == len(csp['nodes']):
            return assignment

        var = select_unassigned_variable(assignment, csp['domains'])
        for value in csp['domains'][var]:
            if is_consistent(var, value, assignment, csp, neighbors):
                assignment[var] = value
                backup_domains = {v: csp['domains'][v][:] for v in csp['nodes']}
                if ac3:
                    _, reduced_domains = AC3(csp, neighbors)
                else:
                    _, reduced_domains = AC4(csp)

                if reduced_domains[var] and backtrack():
                    return assignment
                csp['domains'] = backup_domains
                del assignment[var]
        return None

    return backtrack()


csp = {
    "nodes": nodes,
    "domains": domains,
    "constraints": constraints
}

#AC3:
print("\nBacktracking + AC3 algorithm: \n")
start = time()
solution = backtracking_search(csp, graph, True)
s = time() - start
if solution:
    print("Solution found:", solution)
else:
    print("No solution found!")
print("Solution retrieved in: %.4g seconds" % s)


#AC4:
print("\nBacktracking + AC4 algorithm:\n")
start = time()
solution = backtracking_search(csp, graph, False)
s = time() - start
if solution:
    print("Solution found:", solution)
else:
    print("No solution found!")
print("Solution retrieved in: %.4g seconds" % s)


Backtracking + AC3 algorithm: 

Solution found: {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R'}
Solution retrieved in: 0.001942 seconds

Backtracking + AC4 algorithm:

Solution found: {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R'}
Solution retrieved in: 0.0009968 seconds
